In [1]:
!pip install -U langchain

In [1]:
!pip install langchain_experimental

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.doubanio.com/simple/, https://pypi.org/simple/
   ---------------------------------------- 0.0/177.6 kB ? eta -:--:--
   -- ------------------------------------- 10.2/177.6 kB ? eta -:--:--
   ------ -------------------------------- 30.7/177.6 kB 217.9 kB/s eta 0:00:01
   ------------- ------------------------- 61.4/177.6 kB 365.7 kB/s eta 0:00:01
   --------------- ----------------------- 71.7/177.6 kB 357.2 kB/s eta 0:00:01
   --------------- ----------------------- 71.7/177.6 kB 357.2 kB/s eta 0:00:01
   ------------------------ ------------- 112.6/177.6 kB 364.4 kB/s eta 0:00:01
   ------------------------ ------------- 112.6/177.6 kB 364.4 kB/s eta 0:00:01
   ------------------------ ------------- 112.6/177.6 kB 364.4 kB/s eta 0:00:01
   ------------------------ ------------- 112.6/177.6 kB 364.4 kB/s eta 0:00:01
   ------------------------ ------------- 112.6/177.6 

In [3]:
!pip install faiss-cpu

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.doubanio.com/simple/, https://pypi.org/simple/
  Using cached faiss_cpu-1.8.0-cp312-cp312-win_amd64.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 330.3 kB/s eta 0:00:44
   ---------------------------------------- 0.0/14.5 MB 330.3 kB/s eta 0:00:44
   ---------------------------------------- 0.0/14.5 MB 330.3 kB/s eta 0:00:44
   ---------------------------------------- 0.0/14.5 MB 151.3 kB/s eta 0:01:36
   ---------------------------------------- 0.1/14.5 MB 269.5 kB/s eta 0:00:54
   ---------------------------------------- 0.1/14.5 MB 269.5 kB/s eta 0:00:54
   ---------------------------------------- 0.1/14.5 MB 269.5 kB/s eta 0:00:54

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\http\client.py"

In [11]:
import os
deployment = "gpt-35-turbo"
model = "gpt-35-turbo-16k"
embedding_deployment = "text-embedding-ada-002"
api_version = "2023-05-15"
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = "4ca33e22bb964e08c559f4ea3b69324626ad68a574381de788894deaab9c0fd1"

In [12]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool


In [13]:
# 构造 AutoGPT 的工具集
search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [14]:
from langchain.embeddings import OpenAIEmbeddings

In [15]:
from langchain_openai import AzureOpenAIEmbeddings

# OpenAI Embedding 模型
embeddings_model = AzureOpenAIEmbeddings(azure_endpoint=azure_endpoint,
                                  api_key=api_key, 
                                  api_version=api_version)

In [16]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

# OpenAI Embedding 向量维数
embedding_size = 1536
# 使用 Faiss 的 IndexFlatL2 索引
index = faiss.IndexFlatL2(embedding_size)
# 实例化 Faiss 向量数据库
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## 实例化自主智能体 Auto-GPT

In [17]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI

In [20]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(deployment_name = deployment, 
                      api_version=api_version, 
                      temperature=0, 
                      verbose=True)
agent = AutoGPT.from_llm_and_tools(
    ai_name="Jarvis",
    ai_role="Assistant",
    tools=tools,
    llm=llm,
    memory=vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": 0.8}),# 实例化 Faiss 的 VectorStoreRetriever
)

In [21]:
# 打印 Auto-GPT 内部的 chain 日志
agent.chain.verbose = True

In [22]:
agent.run(["2023年成都大运会，中国金牌数是多少"])



> Entering new LLMChain chain...


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\vectorstores.py:343: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


Prompt after formatting:
System: You are Jarvis, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 2023年成都大运会，中国金牌数是多少


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"title": "File Path", "description

'I have completed all my objectives. Thank you!'